***Opgave 2 – Temperatuur (30 pt)  
De temperatuur per dag hangt af van verschillende factoren.
Toets met α =0 , 05 of de gemiddelde temperatuur per dag afhangt van de
windrichting, de hoeveelheid bewolking en/of het zicht.*** 
* Als je concludeert dat alle groepen gemiddeld genomen even goed
scoren, geef dan een 95% betrouwbaarheidsinterval voor dit
gemiddelde.  
* Als je concludeert dat niet alle groepen gemiddeld genomen even
goed scoren, geef dan aan welke groepen significant beter of juist
slechter scoren dan andere groepen.  

Houd er rekening mee dat sommige variabelen zó veel categorieën
hebben dat er niet goed mee gerekend kan worden (omdat sommige
groepen te klein worden) of dat er niets uit op te maken valt. Het kan dus
nodig zijn om nieuwe variabelen aan te maken waarin (zinvolle)
categorieën zijn samengevoegd.  

regressie
validiteit
anova

In [6]:
# inladen bibliotheken en dataset
import scipy.stats as sp 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as st
import statsmodels.api as sm
import statsmodels.formula.api as sf
execfile('levene.py') 

etmgeg = pd.read_csv("etmgeg2.csv")
etmgeg['salty'] = np.where(df.sodium>400, 
    'yes','no')

sns.boxplot(data=etmgeg, x='type', hue='salty', y='calories')
plt.show()

regmodel=sf.ols('calories ~ type*salty',data=etmgeg)
regres=regmodel.fit()
print(regres.summary())

anovatab=sm.stats.anova_lm(regres,typ=1)
print(anovatab)

FileNotFoundError: [Errno 2] No such file or directory: 'levene.py'